In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SparkDemo").getOrCreate()

df_test = spark.read.csv('data/film.csv', header=True, sep=',')
df_test.show(5)

+-------+----------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+-------------------+
|film_id|           title|         description|release_year|language_id|original_language_id|rental_duration|rental_rate|length|replacement_cost|rating|    special_features|        last_update|
+-------+----------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+-------------------+
|      1|ACADEMY DINOSAUR|An Epic Drama of ...|        2006|          1|                null|              6|       0.99|    86|           20.99|    PG|Deleted Scenes,Be...|2006-02-15 05:03:42|
|      2|  ACE GOLDFINGER|A Astounding Epis...|        2006|          1|                null|              3|       4.99|    48|           12.99|     G|Trailers,Deleted ...|2006-02-15 05:03:42|
|      3|ADAPTATION HOLES|A As

In [8]:
df_test.describe().show()

25/10/06 13:10:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+----------------+--------------------+------------+-----------+--------------------+------------------+------------------+-----------------+------------------+------+--------------------+-------------------+
|summary|          film_id|           title|         description|release_year|language_id|original_language_id|   rental_duration|       rental_rate|           length|  replacement_cost|rating|    special_features|        last_update|
+-------+-----------------+----------------+--------------------+------------+-----------+--------------------+------------------+------------------+-----------------+------------------+------+--------------------+-------------------+
|  count|             1000|            1000|                1000|        1000|       1000|                   0|              1000|              1000|             1000|              1000|  1000|                1000|               1000|
|   mean|            500.5|            null|                

In [10]:
from IPython.display import display
display(df_test)

DataFrame[film_id: string, title: string, description: string, release_year: string, language_id: string, original_language_id: string, rental_duration: string, rental_rate: string, length: string, replacement_cost: string, rating: string, special_features: string, last_update: string]

In [13]:
from pyspark.sql import functions as F

df_test = df_test.withColumn("Title and year",
    F.concat(df_test['title'], F.lit(' '), df_test['release_year'])
)
df_test.show(5)

+-------+----------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+-------------------+--------------------+--------------------+
|film_id|           title|         description|release_year|language_id|original_language_id|rental_duration|rental_rate|length|replacement_cost|rating|    special_features|        last_update|            Fullname|      Title and year|
+-------+----------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+-------------------+--------------------+--------------------+
|      1|ACADEMY DINOSAUR|An Epic Drama of ...|        2006|          1|                null|              6|       0.99|    86|           20.99|    PG|Deleted Scenes,Be...|2006-02-15 05:03:42|ACADEMY DINOSAUR ...|ACADEMY DINOSAUR ...|
|      2|  ACE GOLDFINGER|A Astounding Epis...|        2

In [21]:
df_persons = spark.read.json('data/persons.json')
df_persons.show(10, truncate=False)

+---------------+----+----------+----+---------+
|_corrupt_record|age |city      |id  |name     |
+---------------+----+----------+----+---------+
|[              |null|null      |null|null     |
|null           |30  |Paris     |1   |Alice    |
|null           |25  |Lyon      |2   |Bob      |
|null           |35  |Marseille |3   |Céline   |
|null           |28  |Paris     |4   |David    |
|null           |40  |Bordeaux  |5   |Emma     |
|null           |22  |Nice      |6   |François |
|null           |31  |Strasbourg|7   |Gabrielle|
|null           |27  |Lille     |8   |Hugo     |
|null           |29  |Nantes    |9   |Inès     |
+---------------+----+----------+----+---------+
only showing top 10 rows



In [23]:
# oeioke_no_corrupt_data
df_persons = spark.read.json('data/persons.json')
df_persons.show(10, truncate=False)

df_persons = df_persons.filter(df_persons['_corrupt_record'].isNull())
df_persons.dropna(how= 'all')
df_persons.show(10, truncate=False)

+---------------+----+----------+----+---------+
|_corrupt_record|age |city      |id  |name     |
+---------------+----+----------+----+---------+
|[              |null|null      |null|null     |
|null           |30  |Paris     |1   |Alice    |
|null           |25  |Lyon      |2   |Bob      |
|null           |35  |Marseille |3   |Céline   |
|null           |28  |Paris     |4   |David    |
|null           |40  |Bordeaux  |5   |Emma     |
|null           |22  |Nice      |6   |François |
|null           |31  |Strasbourg|7   |Gabrielle|
|null           |27  |Lille     |8   |Hugo     |
|null           |29  |Nantes    |9   |Inès     |
+---------------+----+----------+----+---------+
only showing top 10 rows

+---------------+---+----------+---+---------+
|_corrupt_record|age|city      |id |name     |
+---------------+---+----------+---+---------+
|null           |30 |Paris     |1  |Alice    |
|null           |25 |Lyon      |2  |Bob      |
|null           |35 |Marseille |3  |Céline   |
|null 

In [25]:
df_persons = spark.read.option('multiline', True).json('data/persons.json')
df_persons.show(10, truncate= False)

+---+----------+---+---------+
|age|city      |id |name     |
+---+----------+---+---------+
|30 |Paris     |1  |Alice    |
|25 |Lyon      |2  |Bob      |
|35 |Marseille |3  |Céline   |
|28 |Paris     |4  |David    |
|40 |Bordeaux  |5  |Emma     |
|22 |Nice      |6  |François |
|31 |Strasbourg|7  |Gabrielle|
|27 |Lille     |8  |Hugo     |
|29 |Nantes    |9  |Inès     |
|33 |Toulouse  |10 |Julien   |
+---+----------+---+---------+
only showing top 10 rows



In [26]:
df_persons = spark.read.json('data/persons.json')

df_persons=df_persons.drop('_corrupt_record')
df_persons=df_persons.dropna(how= 'all')
df_persons.show(10, truncate=False)

+---+----------+---+---------+
|age|city      |id |name     |
+---+----------+---+---------+
|30 |Paris     |1  |Alice    |
|25 |Lyon      |2  |Bob      |
|35 |Marseille |3  |Céline   |
|28 |Paris     |4  |David    |
|40 |Bordeaux  |5  |Emma     |
|22 |Nice      |6  |François |
|31 |Strasbourg|7  |Gabrielle|
|27 |Lille     |8  |Hugo     |
|29 |Nantes    |9  |Inès     |
|33 |Toulouse  |10 |Julien   |
+---+----------+---+---------+
only showing top 10 rows



In [29]:
df_persons.select('id', 'name', 'age').show(10, truncate = False)

+---+---------+---+
|id |name     |age|
+---+---------+---+
|1  |Alice    |30 |
|2  |Bob      |25 |
|3  |Céline   |35 |
|4  |David    |28 |
|5  |Emma     |40 |
|6  |François |22 |
|7  |Gabrielle|31 |
|8  |Hugo     |27 |
|9  |Inès     |29 |
|10 |Julien   |33 |
+---+---------+---+
only showing top 10 rows



In [30]:
df_persons.select('id', 'name', 'age').show(10, truncate = True)

+---+---------+---+
| id|     name|age|
+---+---------+---+
|  1|    Alice| 30|
|  2|      Bob| 25|
|  3|   Céline| 35|
|  4|    David| 28|
|  5|     Emma| 40|
|  6| François| 22|
|  7|Gabrielle| 31|
|  8|     Hugo| 27|
|  9|     Inès| 29|
| 10|   Julien| 33|
+---+---------+---+
only showing top 10 rows



In [ ]:
from pyspark.sql import functions as F

df_persons.select(F.max('age')).show(10)

In [ ]:
df_persons.groupBy('age').count().show()